In [1]:
# A) library importation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl as px
import warnings
from IPython.display import display 
from scipy.stats import skew
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

<class 'ModuleNotFoundError'>: No module named 'seaborn'

In [ ]:
# B) Loading data set 
filepath = "churn_real.xlsx"
df = pd.read_excel(filepath, engine='openpyxl')
# Engine is needed for .xlsx files

# Code to check contents
missing_all = df.isnull().sum().sort_values(ascending=False)
display(missing_all)
print("First 10 rows:")
display(df.head(10))
display(df.columns.tolist())
display(df.describe(include='all'))


In [ ]:
# C) Total Charges numeric conversion
print(df['Total Charges'].dtype)

df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')
#coerce will convert non-numeric values to NaN
print(df['Total Charges'].dtype)

print("Missing values in Total Charges:", df['Total Charges'].isnull().sum())


In [ ]:
# D) Summary statistics visualization
desc = df.describe().drop("count")
#
styled_desc = desc.style\
    .set_properties(**{
        'background-color': 'lightgrey',
        'text-align': 'center',
        'color': 'black'
    })\
    .set_table_styles([
        {
            'selector': 'thead th',
            'props': [
                ('background-color', 'navy'),
                ('color', 'black'),
                ('text-align', 'center')
            ]
        }
    ])

# Display the styled DataFrame
display(styled_desc)

In [ ]:
# E)Customer Churn Anaylsis
relevant_numeric_data = ['Monthly Charges', 'Total Charges', 'Tenure Months','CLTV']
# creates a list of columns of interest

print("Data types of relevant numeric columns:")
print(df[relevant_numeric_data].dtypes)

means = df.groupby('Churn Label')[relevant_numeric_data].mean()
# calculates the mean of the relevant numeric columns grouped by Churn Label                            

print(df[relevant_numeric_data].dtypes)

n_cols = 2
n_rows = (len(relevant_numeric_data) + 1) // n_cols
# calculates the number of rows needed for the subplots

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, n_rows * 4))
axes = axes.flatten()
# flattens the axes array to make it easier to iterate over

for i, col in enumerate(relevant_numeric_data):
    ax = axes[i]
    sns.barplot(x=means.index, y=means[col], ax=ax, palette="Set2")
    
    ax.set_title(f'Average {col} by Churn Label', fontsize=12)
    ax.set_xlabel("Churn Label")
    ax.set_ylabel(f'Mean {col}')
    
    for p in ax.patches:
        value = p.get_height()
        ax.annotate(f'{value:.1f}', (p.get_x() + p.get_width()/2, value + 0.5),
                    ha='center', fontsize=10)
        
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])
    # delete unused subplots

plt.tight_layout()
# Adjust layout to prevent overlap
plt.show()
    

In [ ]:
# G) Churn Distribution Visualization
churn_counts = df['Churn Label'].value_counts()

# Pie chart settings
colors = ['#1f3b73', '#ff7f0e']
explode = (0.05, 0.05)  # slightly pull apart the slices

plt.figure(figsize=(6, 6))
plt.pie(churn_counts, labels=churn_counts.index, autopct='%1.1f%%',
        startangle=90, colors=colors, explode=explode, shadow=True)
plt.title("Customer Churn Distribution")
plt.axis('equal')  # Equal aspect ratio ensures the pie is circular.
plt.show()

In [ ]:
# H)Variable vs Churn Correlation Analysis
exclude_cols = ['CustomerID', 'Lat long', 'Zip Code', 'Churn Reason', 'Churn Label', 'Country', 'State']
categorical_cols = df.select_dtypes(include='object').columns
categorical_cols = [col for col in categorical_cols if col not in exclude_cols]

n_cols = 2
n_rows = (len(categorical_cols) + 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 5))
axes = axes.flatten()

for i, col in enumerate(categorical_cols):
    ax = axes[i]
    
    # Top 5 most common values in this column
    top_5 = df[col].value_counts().nlargest(5).index
    df_top = df[df[col].isin(top_5)].copy()
    
    # ✅ Use df_top for cleaner charts
    sns.countplot(data=df_top, x=col, hue='Churn Label', ax=ax)
    ax.set_title(f'{col} vs Churn', fontsize=12)
    ax.set_xlabel(col)
    ax.set_ylabel("Count")
    ax.tick_params(axis='x', rotation=45)

    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{int(height)}',
                    (p.get_x() + p.get_width() / 2., height + 1),
                    ha='center', fontsize=9)

    ax.legend(title="Churn")

# Clean up unused plots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
# I) Churn Reason Analysis
# Step 1: Count churn reasons
reason_counts = df['Churn Reason'].value_counts()

# Step 2: Calculate percentages
reason_percent = df['Churn Reason'].value_counts(normalize=True) * 100

# Step 3: Combine into one DataFrame
churn_summary = pd.DataFrame({
    'Count': reason_counts,
    'Percentage': reason_percent
})

# Step 4: Format percentages to 1 decimal place with %
churn_summary['Percentage'] = churn_summary['Percentage'].map("{:.1f}%".format)

churn_summary.index.name = 'Churn Reason'
churn_summary.reset_index(inplace=True)

# Display result
display(churn_summary)


In [ ]:
#j) Churn Reason Evaluation
# As seen by the screenshot provided above the top 3 churn reasons are as follows:
# - Attitude of support Person
# - Competitors offering higher download speed
# - Competitors offering more data

# Suggested mitigation strategies or changes would be:
# -To train support personnel on human management skills as well as communication skills as soon as possible 
# since the churn percentage is the highest at 10.3% which is quite significant for an issue that can be resolved 
# relatively quickly, or implement an automatic response system that has been trained to deal with customers 
# effectively and fast.
# -As for dealing with competitors offering higher download speed, it would be advisable to outperform competitors 
# in this aspect or match them, if not improve or strengthen other aspects such as reliability and/or signal 
# connectivity to keep customers happy. Some may prefer quality/functionality over quantity.
# -Lastly, competitors offering more data, this can be mitigated by offering customers equal levels of data but 
# at a much lower price or monthly discounts on data packages. 



In [ ]:
##PART_B

In [ ]:
# A)Missing values Identification

# Display all variables and their missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]  # Filter only variables with missing values
print("Variables with missing values:")
display(missing_values)

In [ ]:
# B) Replacing missing values with mode
TOTAL_CHARGES_MODE= df['Total Charges'].mode()[0]
print("Total Charges Mode:", TOTAL_CHARGES_MODE)
df['Total Charges'].fillna(TOTAL_CHARGES_MODE, inplace=True)
print("Missing values in Total Charges:", df['Total Charges'].isnull().sum())

In [ ]:
# C) Skewness Calculation and Plotting

# Columns to exclude
exclude = ['Latitude', 'Longitude', 'Churn Value', 'Churn Score','Zip Code', 'CustomerID', 'Count']

# Get numeric columns excluding the above
numeric_cols = df.select_dtypes(include='number').columns
numeric_filtered = [col for col in numeric_cols if col not in exclude]

# Calculate skewness
skewness = df[numeric_filtered].skew().sort_values(ascending=False)
skew_df = pd.DataFrame(skewness, columns=['Skewness'])

display(skew_df)

# Plot regression for each variable against 'Churn Score'
for col in numeric_filtered: # fetches the columns that are not in the exclude list
    plt.figure(figsize=(6, 4))
    sns.regplot(x=col, y='Churn Score', data=df, scatter_kws={'alpha': 0.5}, line_kws={'color': 'red'})
    plt.title(f'Regression Plot: {col} vs Churn Score')
    plt.xlabel(col)
    plt.ylabel('Churn Score')
    plt.tight_layout()
    plt.show()

for col in numeric_filtered: # fetches the columns that are not in the exclude list
    plt.figure(figsize=(6, 4))
    for label in df['Churn Label'].unique():
        sns.kdeplot(df[df['Churn Label'] == label][col], label=label, shade=True)
    plt.title(f'KDE Plot of {col} by Churn Label')
    plt.xlabel(col)
    plt.ylabel('Density')
    plt.legend()
    plt.tight_layout()
    plt.show()

for col in numeric_filtered: # fetches the columns that are not in the exclude list
    plt.figure(figsize=(6, 4))
    sns.boxplot(x='Churn Label', y=col, data=df)
    plt.title(f'Box Plot of {col} by Churn Label')
    plt.xlabel('Churn Label')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()


In [ ]:
#PART C
# HYPOTHESIS TESTING
# 1. Phone Service vs Churn Label
# The null hypothesis is that there is no relationship between the phone service and churn label.
# The alternative hypothesis is that there is a relationship between the phone service and churn label.


contingency_table = pd.crosstab(df['Phone Service'], df['Churn Label'])

# Perform chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Contingency Table:")
display(contingency_table)

print(f"\nChi-square Statistic: {chi2:.4f}")
print(f"Degrees of Freedom: {dof}")
print(f"P-value: {p:.4f}")


In [ ]:
# 2. Contract vs Churn Label
# The null hypothesis is that there is no relationship between the contract type and churn label.
# The alternative hypothesis is that there is a relationship between the contract type and churn label.

contract_table = pd.crosstab(df['Contract'], df['Churn Label'])

# Chi-square test
chi2_contract, p_contract, dof_contract, expected_contract = chi2_contingency(contract_table)

# Display Results
display(contract_table)
print(f"Chi-square Statistic: {chi2_contract:.4f}")
print(f"Degrees of Freedom: {dof_contract}")
print(f"P-value1: {p_contract:.2e}")

if p_contract < 0.05:
    print("P-value < 0.05 → Reject the null hypothesis: There IS a relationship.")
else:
    print("P-value > 0.05 → Fail to reject the null hypothesis: NO significant relationship.")

In [ ]:
# 3. Senior Citizen vs Churn Label
# The null hypothesis is that there is no relationship between the senior citizen and churn label.
# The alternative hypothesis is that there is a relationship between the senior citizen and churn label.

# Contingency Table
senior_table = pd.crosstab(df['Senior Citizen'], df['Churn Label'])

# Chi-square test
chi2_senior, p_senior, dof_senior, expected_senior = chi2_contingency(senior_table)

# Display Results
display(senior_table)
print(f"Chi-square Statistic: {chi2_senior:.4f}")
print(f"Degrees of Freedom: {dof_senior}")
print(f"P-value: {p_senior:.2e}")

if p_senior < 0.05:
    print("P-value < 0.05 → Reject the null hypothesis.")
else:
    print("P-value > 0.05 → Fail to reject the null hypothesis.")


In [ ]:
# PART D

# Step 1: Select numeric features (excluding non-relevant ones)
features = ['Monthly Charges', 'Total Charges', 'Tenure Months', 'CLTV']
X = df[features]

# Step 2: Initialize the scaler
scaler = StandardScaler()

# Step 3: Apply the scaler
X_scaled = scaler.fit_transform(X)

# Turn back into a DataFrame
X_scaled_df = pd.DataFrame(X_scaled, columns=features)

# Preview
X_scaled_df.head()

X_scaled_df.describe().round(3)

In [ ]:
# Splitting the data into training and testing sets
y = df['Churn Label']

X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state= 50, stratify=y)

X_train.shape



In [ ]:
# Checking the shape of the training and testing sets
X_test.shape

In [ ]:
# Checking the shape of the training and testing sets
y_train.shape

In [ ]:
# Checking the shape of the training and testing sets
y_test.shape

In [ ]:
# Fitting the model
model = LogisticRegression(solver='liblinear', random_state=50)
model.fit(X_train, y_train)

In [ ]:
# Predicting the test set results
y_pred = model.predict(X_test)

# Checking the accuracy of the model
model.score(X_test, y_test)

In [ ]:
# Checking the classification report
print(classification_report(y_test, y_pred))


In [ ]:
# Checking the confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix - Logistic Regression(Churn)")
plt.show()